In [1]:
from seleniumwire import webdriver  # Import from seleniumwire
from seleniumwire.utils import decode
import re, time
from selenium.webdriver.common.by import By
import pandas as pd
import os, re, time, json, random
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException


In [2]:
driver = webdriver.Chrome(r'C:\Users\USER\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe')
driver.implicitly_wait(10)


In [3]:
mobile = '9910737626'
login_url = 'https://app.thomsondigital2021.com/login'
mobile_email_xp = '//*[@id="mobile_num"]'
next_btn_xp = '//*[@id="mobile-input-btn"]'
enter_otp_xp = '//*[@id="otp_num"]'
verify_xp = '//*[@id="otp-verify-btn"]'
custom_url = 'https://app.thomsondigital2021.com/exam_custom'

sub_xp = '//*[@id="Mathematics_btn"]'
save_and_next_xp_f = '//*[@id="question_section"]/div[3]/a[1]'
save_and_next_xp_o = '//*[@id="question_section"]/div/div[3]/a[1]'
q_type_xp = '//*[@id="question_section"]/div/div[1]/span'
sub_test_xp = '//*[@id="bt-modal-confirm_over"]'
endExam_xp = '//*[@id="endExam"]'

wait = 3
prob = 1 # for toppers


In [4]:
global q_ids
q_ids = []

file_path = os.path.join( os.getcwd(), 'new\\all_live_questions.xlsx')
q_data = pd.read_excel(file_path)
# q_data[:1]

In [5]:

# Login 
driver.get(login_url)
driver.find_element(By.XPATH, mobile_email_xp).send_keys(mobile)
# time.sleep(1)
next_btn_elm = driver.find_element(By.XPATH, next_btn_xp)
driver.execute_script("arguments[0].click();", next_btn_elm)
time.sleep(5)

otp = []
otp_url = 'https://app.thomsondigital2021.com/sendotplogin'
for request in driver.requests:
    if request.response:
        if request.url == otp_url:
            test_str = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
            test_str = test_str.decode('UTF-8')
            otp += re.findall(r'(\d+)', test_str)
otp = otp[-1]      
enter_otp_elm = driver.find_element(By.XPATH, enter_otp_xp).send_keys(otp)
time.sleep(1)
verify_elm = driver.find_element(By.ID, 'otp-verify-btn')
driver.execute_script("arguments[0].click();", verify_elm) 


In [6]:

def get_corr_options_or_ans(q_id, prob):
    isItNumQuestion = int(q_data[q_data['question_id']==q_id]['template_type'].values[0])==11
    if isItNumQuestion:
        ans = re.sub('[\[\]\"]', '', q_data[q_data['question_id']==q_id]['answers'].values[0])
        if random.random() > prob:
            ans += 1
        return ans
    else:
        ans_text = q_data[q_data['question_id']==q_id]['answers'].values[0]
        ans_text_dict = json.loads(ans_text)
        corr_options = list(ans_text_dict.keys())
        corr_options = [int(i) for i in corr_options]
        incorr_options = [i for i in range(1,5) if i not in corr_options]
        if random.random() > prob:
            corr_options[0] = incorr_options[0]
        return corr_options



In [7]:

def do_question_num(save_and_next_xp, wait, curr_q_id):
    ans = get_corr_options_or_ans(curr_q_id, prob)
    num_ans_box_xp = f'//*[@id="quest_option_{curr_q_id}"]'
    num_ans_box_elm = driver.find_element(By.XPATH, num_ans_box_xp)
    num_ans_box_elm.clear()
    num_ans_box_elm.send_keys(ans)
    time.sleep(2)
    # num_ans_box_elm.clear()
    save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
    # time.sleep(1)
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    return 


In [8]:

def get_curr_q_id():
    curr_q_xp = '//*[@id="current_question"]'
    curr_q_elm = driver.find_element(By.XPATH, curr_q_xp)
    curr_q_id = curr_q_elm.get_attribute('value')    
    return int(curr_q_id)
# get_curr_q_id()


In [9]:

def do_question(o_xp_format, save_and_next_xp, wait):
    curr_q_id = get_curr_q_id()
    corr_options = get_corr_options_or_ans(curr_q_id, prob)
    for corr_option in corr_options:
        o_xp = o_xp_format.replace('{corr_option}', f'{corr_option}')
        o_xp_elm = driver.find_element(By.XPATH, o_xp)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", o_xp_elm)
    # click save and next after wait time is over
    time.sleep(0.5)

    save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    
    time.sleep(2)
    endExam_elm = driver.find_element(By.XPATH, endExam_xp)
    curr_id_temp = get_curr_q_id()
    
    if curr_id_temp == curr_q_id and endExam_elm.value_of_css_property('display') == 'none':
        try:
            do_question(o_xp_format, save_and_next_xp, wait)
        except:
            input(f'Please do the question manually {corr_options}')
    return


In [10]:
# q_ids
def get_subject_and_topic_from_q_id(curr_q_id):
    curr_df = q_data[q_data['question_id']==curr_q_id]
    subject_id = curr_df['subject_id'].values[0]
    topic_id = curr_df['topic_id'].values[0]
    
    return subject_id, int(topic_id)


In [11]:
# Attempt Topic test
o_xp_o_format = '//*[@id="question_section"]/div/div[2]/div[2]/div[{corr_option}]/div/label'
o_xp_f_format = '//*[@id="question_section"]/div[2]/div[2]/div[{corr_option}]/div/label'

def do_topic_test():
    
    # Do first question
    curr_q_id = get_curr_q_id()
    subject_id, topic_id = get_subject_and_topic_from_q_id(curr_q_id)
    q_ids.append(curr_q_id)
    q_type_elm = driver.find_element(By.XPATH, q_type_xp)
    q_type = q_type_elm.get_attribute('innerText')
    try:
        if q_type == 'NUMERICAL' or int(q_data[q_data['question_id']==curr_q_id]['template_type'].values[0])==11:
            do_question_num(save_and_next_xp_f, wait, curr_q_id)
        else:
            do_question(o_xp_f_format, save_and_next_xp_f, wait)
    except StaleElementReferenceException:
        pass
        

    # Do other questions
    sub_test_elm = driver.find_element(By.XPATH, sub_test_xp)
    test_end_condition = sub_test_elm.get_attribute('innerText')
    while test_end_condition != 'SUBMIT TEST':
        
        curr_q_id = get_curr_q_id()
        q_ids.append(curr_q_id)
        q_type_elm = driver.find_element(By.XPATH, q_type_xp)
        q_type = q_type_elm.get_attribute('innerText')
        try:
            if q_type == 'NUMERICAL' or int(q_data[q_data['question_id']==curr_q_id]['template_type'].values[0])==11:
                do_question_num(save_and_next_xp_o, wait, curr_q_id)
            else:
                do_question(o_xp_o_format, save_and_next_xp_o, wait)
                
            test_end_condition = sub_test_elm.get_attribute('innerText')
        # except StaleElementReferenceException:
        except:
            test_end_condition = sub_test_elm.get_attribute('innerText')
                
    time.sleep(1)
    # end_exam_elm = driver.find_element(By.XPATH, sub_test_xp)
    SUB_TEST_elm = driver.find_element(By.ID, 'bt-modal-confirm_over')
    time.sleep(1)
    driver.execute_script("arguments[0].click();", SUB_TEST_elm)
    driver.save_screenshot(os.path.join(os.getcwd(),'botTestOutput',f'subject_id-{subject_id}  topic_id-{topic_id}.png'))
    
    return 


In [12]:

def start_topic_test(topic_id):
    
    topic_select_elm = driver.find_element(By.ID, f"chpt_topic_{topic_id}")
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", topic_select_elm)
    take_test_for_sel_topic_elm = driver.find_element(By.XPATH, '//*[@id="topic_custom_footer"]/button')
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", take_test_for_sel_topic_elm)
    time.sleep(1)
    go_for_it_elm = driver.find_element(By.XPATH, '//*[@id="goto-exam-btn"]')
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", go_for_it_elm)

    test_instruction_elm = driver.find_element(By.ID, 'test_instruction')
    driver.execute_script("arguments[0].click();", go_for_it_elm)
    time.sleep(0.5)
    while test_instruction_elm.is_displayed():
        driver.execute_script("arguments[0].click();", go_for_it_elm)
        test_instruction_elm = driver.find_element(By.ID, 'test_instruction')
        print('At instruction page')
        time.sleep(0.5)
    return


In [13]:

def get_topic_ids(chp_id):
    time.sleep(1)
    chap_box_content = driver.find_element(By.ID, f'chapter_box_{chp_id}').get_attribute('innerHTML')
    topic_ids = re.findall(r'id="chpt_topic_(\d+)"', chap_box_content)
    topic_ids = [int(i) for i in topic_ids]
    return topic_ids


In [14]:

def expand_chapter(chp_id):
    driver.get(custom_url)
    time.sleep(1)
    chp_expand_elm = driver.find_element(By.ID, f"expandTopic_{chp_id}")
    time.sleep(1)
    driver.execute_script("arguments[0].click();", chp_expand_elm)
    time.sleep(0.5)
    return 


In [15]:
def get_chap_ids(subject):
    
    driver.get(custom_url)
    subject_list_innerHTML = driver.find_element(By.ID,f'{subject}_list').get_attribute('innerHTML')
    
    chp_ids = re.findall(r'id="chapter_box_(\d+)"', subject_list_innerHTML)
    chp_ids = [int(i) for i in chp_ids]
    return chp_ids

subjects = ['Mathematics', 'Physics', 'Chemistry']
math_chp_ids = get_chap_ids(subjects[0])
phy_chp_ids = get_chap_ids(subjects[1])
chem_chp_ids = get_chap_ids(subjects[2])


In [ ]:
# Adaptive test starts


chp_no = 0
for chp_id in math_chp_ids[:10]:
    time.sleep(0.5)
    expand_chapter(chp_id)
    time.sleep(0.5)
    topic_ids = get_topic_ids(chp_id)
    topic_no = 0
    for topic_id in topic_ids:
        try:
            start_topic_test(topic_id)
            do_topic_test()
        except:
            pass
        expand_chapter(chp_id)
        topic_no+=1
        print(f'Topics completed: {topic_no}')

    chp_no+=1
    print(f'Chapters completed: {chp_no}')
  

Please do the question manually [1] 


Topics completed: 1
Topics completed: 2
Topics completed: 3
Chapters completed: 1
Topics completed: 1


Please do the question manually [1] 


Topics completed: 2
Topics completed: 3


Please do the question manually [3] 


...

Topics completed: 3
Topics completed: 4
Topics completed: 5
Chapters completed: 4
Topics completed: 1
Topics completed: 2
Topics completed: 3
Topics completed: 4
Topics completed: 5
Topics completed: 6
Topics completed: 7
Chapters completed: 5
Topics completed: 1
Topics completed: 2
Topics completed: 3
Topics completed: 4
Topics completed: 5
Chapters completed: 6
Topics completed: 1


Please do the question manually [3] 


Topics completed: 2
Topics completed: 3
Chapters completed: 7
Topics completed: 1


Please do the question manually [1] 


Topics completed: 2


In [ ]:
# For capturing network activity

for request in driver.requests:
    if request.response and request.url == 'https://app.qandi.com/saveAdaptiveAnswer':
        print(request.url, request.body, decode(request.response.body), decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity')))
